### Libraries Importing

In [1]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv
from urllib.parse import urlparse, quote_plus
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error




### feature loading

In [2]:

load_dotenv()
mongo_uri = os.getenv("MONGO_URI")

parsed = urlparse(mongo_uri)
if parsed.username:
    escaped_user = quote_plus(parsed.username)
    escaped_pass = quote_plus(parsed.password) if parsed.password else ''
    netloc = f"{escaped_user}:{escaped_pass}@{parsed.hostname}"
    if parsed.port:
        netloc += f":{parsed.port}"
    mongo_uri = parsed._replace(netloc=netloc).geturl()

client = MongoClient(mongo_uri)

db = client["aqi_feature_store"]
collection = db["hourly_features"]

data = list(collection.find({}, {"_id": 0}))
df = pd.DataFrame(data)



In [3]:
df.columns

Index(['timestamp', 'seasons', 'hour', 'month', 'year', 'day_of_week',
       'timeof_day', 'aqi_lag_1', 'aqi_lag_2', 'aqi_lag_3', 'aqi_lag_6',
       'aqi_lag_12', 'aqi_lag_24', 'aqi_24hr_avg', 'aqi'],
      dtype='object')

###  Create 72-step target (direct multi-output)

In [5]:
HORIZON = 72

y = pd.concat(
    [df["aqi"].shift(-i) for i in range(1, HORIZON + 1)],
    axis=1
)
y.columns = [f"aqi_t+{i}" for i in range(1, HORIZON + 1)]

X = df.drop(columns=["aqi", "timestamp"])

X = X.iloc[:-HORIZON]
y = y.iloc[:-HORIZON]


### time based train test split

In [6]:
split = int(len(X) * 0.8)

X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]


### Train XGBoost direct multi-output model

In [7]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgboost_model = MultiOutputRegressor(
    XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42
    )
)

xgboost_model.fit(X_train, y_train)


,estimator,"XGBRegressor(...ree=None, ...)"
,n_jobs,None
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None


### Predicting AQI

In [8]:
y_pred_72 = xgboost_model.predict(X_test.iloc[[-1]])


In [9]:
y_pred= xgboost_model.predict(X_test)


### Convert to 3-day AQI for UI

In [10]:
import numpy as np

day1 = np.mean(y_pred_72[0][:24])
day2 = np.mean(y_pred_72[0][24:48])
day3 = np.mean(y_pred_72[0][48:72])

print(day1, day2, day3)


4.2960277 3.9002144 3.5139115


### Performance Evaluation

In [11]:
# Overall metrics (all 72 hours)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Overall MAE  : {mae:.3f}")
print(f"Overall RMSE : {rmse:.3f}")
print(f"Overall MAPE : {mape:.2f}%")

# # Daily metrics (24h blocks)
# for day in range(3):
#     start = day * 24
#     end = (day + 1) * 24

#     day_mae = mean_absolute_error(y_test[:, start:end], y_pred_72[:, start:end])
#     day_rmse = np.sqrt(mean_squared_error(y_test[:, start:end], y_pred_72[:, start:end]))

#     print(f"Day {day+1} MAE  : {day_mae:.3f}")
#     print(f"Day {day+1} RMSE : {day_rmse:.3f}")


Overall MAE  : 0.521
Overall RMSE : 0.759
Overall MAPE : 10.74%


### Ransom Forest Regressor Training

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=200))
rf_model.fit(X_train, y_train)


,estimator,RandomForestR...stimators=200)
,n_jobs,None
,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0


### Prediction

In [14]:
y_pred = rf_model.predict(X_test) 

In [16]:
y_pred_72 = rf_model.predict(X_test.iloc[[-1]])

In [17]:
import numpy as np

day1 = np.mean(y_pred_72[0][:24])
day2 = np.mean(y_pred_72[0][24:48])
day3 = np.mean(y_pred_72[0][48:72])

print(day1, day2, day3)


4.34921130952381 3.6593750000000003 3.0570833333333334


### Performace Evaluation

In [18]:

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Overall MAE  : {mae:.3f}")
print(f"Overall RMSE : {rmse:.3f}")
print(f"Overall MAPE : {mape:.2f}%")


Overall MAE  : 0.664
Overall RMSE : 1.012
Overall MAPE : 13.54%


## CNN Model 

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [33]:
import torch
import torch.nn as nn

class AQIMLP(nn.Module):
    def __init__(self, input_dim, output_dim=72):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)


In [35]:
mlp_model = AQIMLP(input_dim=X_train_scaled.shape[1])

Xtr = torch.tensor(X_train_scaled, dtype=torch.float32)
ytr = torch.tensor(y_train.values, dtype=torch.float32)

optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(50):
    optimizer.zero_grad()
    preds = mlp_model(Xtr)
    loss = criterion(preds, ytr)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 15.6852
Epoch 10, Loss: 14.1242
Epoch 20, Loss: 10.3723
Epoch 30, Loss: 4.5300
Epoch 40, Loss: 1.7999


1. Prediction on test set

In [36]:
mlp_model.eval()
with torch.no_grad():
    Xte = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_pred_72 = mlp_model(Xte).numpy()


2. Convert 72-hour prediction → 3-day AQI (daily average)

In [37]:
import numpy as np

# average each 24-hour block
day1_avg = y_pred_72[:, 0:24].mean(axis=1)
day2_avg = y_pred_72[:, 24:48].mean(axis=1)
day3_avg = y_pred_72[:, 48:72].mean(axis=1)

daily_aqi_pred = np.vstack([day1_avg, day2_avg, day3_avg]).T


In [38]:
daily_aqi_pred_rounded = np.clip(
    np.round(daily_aqi_pred), 2, 5
)


In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_true = y_test.values

mae = mean_absolute_error(y_true, y_pred_72)
rmse = np.sqrt(mean_squared_error(y_true, y_pred_72))
mape = np.mean(np.abs((y_true - y_pred_72) / y_true)) * 100

print(f"Overall MAE  : {mae:.3f}")
print(f"Overall RMSE : {rmse:.3f}")
print(f"Overall MAPE : {mape:.2f}%")


Overall MAE  : 1.789
Overall RMSE : 1.878
Overall MAPE : 36.97%
